In [1]:
import os
import sys
import numpy as np
import pandas as pd
import csv


In [7]:
data_dir = "/mnt/c/Users/ruotong/Home/xERTE/tKGR/data/ml-1m"
print(os.listdir(data_dir))
df = pd.read_csv(os.path.join(data_dir, "ratings.dat"), sep="::",header=None, names=['user','movie','rating','timestamp'])
print(df.head())
df=df.reindex(columns=['user','rating','movie','timestamp'])
df.head()

SyntaxError: closing parenthesis ']' does not match opening parenthesis '(' (<ipython-input-7-7939dfd55601>, line 5)